In [1]:
import pandas as pd
import numpy as np
import os

#### SELECT CURRENT PATH

In [2]:
import os
cwd = os.path.abspath('')
print(cwd)

C:\Users\kitit\Desktop\Folder\AJ. Prasert\JOB1\DEMO


#### SELECT FILE .XLSX IN CURRENT FOLDER
ถ้าอยากให้มี xls แก้เพิ่ม or filename.endswith('.xls')

In [3]:
for file in os.listdir(cwd):
    filename = os.fsdecode(file)
    if filename.endswith('.xlsx'):
        print(filename)
    else:
        pass

1.xlsx
2.xlsx
3.xlsx
4.xlsx
5.xlsx


#### CREATE DATAFRAME FROM SELECTED FILE

ใช้ loop สร้าง dataframe ใน dict
<br>
โดย key ชื่อว่า df1, df2, df3, ...
<br>
    value เป็น dataframe

In [4]:
dict_store_df = {}
for file in os.listdir(cwd):
    filename = os.fsdecode(file)
    if filename == 'result.xlsx':
        pass
    elif filename.endswith('.xlsx'):
        dict_store_df['df{}'.format(filename[0])] = pd.read_excel(filename, skiprows= 12)
    else:
        pass

#### FILTER TWO COLUMN ('TITLE', 'CITATIONS') AND DROP 2 LAST ROWS
<br>
filter ให้เหลือแค่คอลัมน์ Title, Citations
<br>
ลบสองแถวสุดท้าย เพราะว่าไม่มีข้อมูล

In [5]:
for key in dict_store_df:
    dict_store_df[key] = dict_store_df[key][['Title', 'Citations']]
    dict_store_df[key].drop(dict_store_df[key].tail(2).index,inplace=True)

#### FIND A MONTH NAME
จาก format ไฟล์ excel ตัววันที่จะอยู่แถวที่ 9 คอลัมน์ C

In [6]:
month = pd.read_excel('2.xlsx', skiprows= 6)
month = month.iloc[1,2]

# CREATE  RESULT DATAFRAME

In [7]:
result = pd.merge(dict_store_df['df2'], dict_store_df['df1'], how = 'left', on = 'Title')
result['change'] = result['Citations_x'] - result['Citations_y']
result = result[['Title', 'Citations_y','change']]
result.columns = ['Title', 'First', month]
result['Total'] = result['First'] + result[month]
result.head()

,Title,First,1 March 2019,Total
0,Auditor Independence and Audit Quality: A Lite...,40.0,0.0,40.0
1,A novel architecture for requirement-oriented ...,35.0,0.0,35.0
2,Compliance checking for requirement-oriented s...,34.0,0.0,34.0
3,The societal relevance of management accountin...,30.0,0.0,30.0
4,Revenue maximization with a single sample,27.0,0.0,27.0


## LOOP ADD DIFF COLUMN TO RESULT DATAFRAME
ใช้ loop โดยการ left join (result, df) จากนั้นหาผลต่าง ทำไปเรื่อยๆจ้า

In [8]:
for i in range(3,len(dict_store_df)+1):
    month = pd.read_excel('{}.xlsx'.format(i), skiprows= 6)
    month = month.iloc[1,2]
    result = pd.merge(dict_store_df['df{}'.format(i)], result, how='left', on = 'Title')
    result.fillna(0, inplace = True)
    result[month] = result['Citations'] - result['Total']
    del result['Total']
    del result['Citations']
    result['Total'] = result.iloc[:, 1:].sum(axis = 1)


month = pd.read_excel('3.xlsx', skiprows= 6)
month = month.iloc[1,2]
result = pd.merge(dict_store_df['df3'], result, how='left', on = 'Title')
result
result[month] = result['Citations'] - result['Total']
del result['Total']
del result['Citations']
result['Total'] = result.iloc[:, 1:].sum(axis = 1)
result

### RESET INDEX

In [9]:
result.index = range(1, len(result)+1)

# DATAFRAME TO EXCEL

In [10]:
result.to_excel('result.xlsx')

# ตกแต่ง excel กันเถอะ

writer = pd.ExcelWriter('result.xlsx', engine='xlsxwriter')
result.to_excel(writer, sheet_name = 'raw')
workbook  = writer.book
worksheet = writer.sheets['raw']
worksheet.set_column('A:A', 5)
worksheet.set_column('B:B', 50)
worksheet.set_column('C:Z', 15)
writer.save()